In [9]:
# 1 persona, 1 ascensor
# estado_ini = (num_personas, persona0,..., personaN, ascensores...)
estado_ini = (
    (1,), (0,)
)#(1, 3, 3)

# estado_goal
estado_goal = (
    (3,), (3,)
)#(1, 2, 2)

# Acciones
# Llamar ascensor (mueve un ascensor vacio a una planta)
# Subir persona (sube una persona dentro de un ascensor a una planta) (comprobar si ascensor y persona estan en misma planta)
# Bajar persona (baja una persona dentro de un ascensor a una planta) (comprobar si ascensor y persona estan en misma planta)

In [10]:
# importamos las cosas que vamos a usar de aima
from search import *

class Ascensores(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal):
        '''Inicializacion de nuestro problema.'''
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        #p = estado_ini[1:estado_ini[0]+1]
        #a = estado_ini[estado_ini[0] + 1:]
        p = list(estado[0])
        a = list(estado[1])
        accs = list()
        if p[0] != a[0]:
            accs.append(("Llamar", str(p[0])));
        if p[0] == a[0] and int(p[0]) > 0:
            accs.append(("Bajar", str(int(a[0])-1)))
        if p[0] == a[0] and int(p[0]) < 12:
            accs.append(("Subir", str(int(a[0])+1)))
        return accs
    
    def result(self, estado, accion):
        #p = list(estado_ini[1:estado_ini[0]+1])
        #a = list(estado_ini[estado_ini[0] + 1:])
        p = list(estado[0])
        a = list(estado[1])
        if accion[0] == "Llamar":
            a[0] = int(accion[1])
        elif accion[0] == "Bajar":
            a[0] = int(accion[1])
            p[0] = int(accion[1])
        elif accion[0] == "Subir":
            a[0] = int(accion[1])
            p[0] = int(accion[1])
        #res = (p + a)
        #res.insert(0, len(p))
        return (tuple(p), tuple(a))


In [11]:
p = Ascensores(estado_ini, estado_goal)
p.initial

((1,), (0,))

In [12]:
p.actions(p.initial)

[('Llamar', '1')]

In [13]:
p.result(p.initial, ("Llamar", 1))

((1,), (1,))

In [6]:
p.goal

((3,), (3,))

In [7]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [8]:
breadth_first_graph_search(p).solution()

[('Llamar', '1'), ('Subir', '2'), ('Subir', '3')]